In [1]:
import os
from PIL import Image
import pandas as pd

In [2]:
df_path = "/Users/user/Documents/Coding/geo/notebook/7-11 Location for Ford.xlsx"
df = pd.read_excel(df_path)
df.head()

,store_id,prov_namt,latitude,longitude
0,1025,กรุงเทพมหานคร,13.838724,100.575318
1,1026,ปทุมธานี,14.036545,100.733520
2,1028,ตรัง,7.567873,99.614094
3,1032,กรุงเทพมหานคร,13.719625,100.442061
4,1033,กรุงเทพมหานคร,13.768390,100.721288


In [3]:
import numpy as np

# add new column name embedding with nan
df["embedding"] = [[]] * len(df)
df.head()

,store_id,prov_namt,latitude,longitude,embedding
0,1025,กรุงเทพมหานคร,13.838724,100.575318,[]
1,1026,ปทุมธานี,14.036545,100.733520,[]
2,1028,ตรัง,7.567873,99.614094,[]
3,1032,กรุงเทพมหานคร,13.719625,100.442061,[]
4,1033,กรุงเทพมหานคร,13.768390,100.721288,[]


In [4]:
# check is store_id is unique
df["store_id"].is_unique

True

In [5]:
# do model inference

In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print(
            "MPS not available because the current PyTorch install was not "
            "built with MPS enabled."
        )
    else:
        print(
            "MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine."
        )

else:
    device = torch.device("mps")
device

device(type='mps')

In [7]:
import timm
from geo_utils import crop_middle_image_from_side_size, apply_circle_mask

model = timm.create_model(
    "maxvit_tiny_tf_512.in1k",
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
).to(
    device
)  # Move the model to the GPU
model = model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)


def preprocess_image(image: Image.Image) -> Image.Image:
    image = crop_middle_image_from_side_size(image, 2400, 1600)
    image = apply_circle_mask(image)
    return image


preprocessed_image_folder = "/Users/user/Documents/Coding/geo/notebook/data/crop_image"


def predict(model, transforms, image_path, preprocessed_image_path, device=device):
    image = Image.open(image_path).convert("RGB")
    preprocessed_image = preprocess_image(image)
    image_tensor = (
        transforms(preprocessed_image).unsqueeze(0).to(device)
    )  # Move input data to the GPU
    output = model(image_tensor)

    # or equivalently (without needing to set num_classes=0)
    output = model.forward_features(image_tensor)
    output = model.forward_head(output, pre_logits=True)
    embedding = output.detach().cpu().numpy()[0]
    preprocessed_image.save(preprocessed_image_path)
    # save preprocessed_image
    return embedding  # Move the result back to the CPU for further processing if needed

/Users/user/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
image_path = "/Users/user/Documents/Coding/geo/notebook/data/full_image/1025.png"
image_path = "/Users/user/Documents/Coding/geo/notebook/data/full_image/1026.png"
preprocessed_image_path = os.path.join(
    preprocessed_image_folder, os.path.basename(image_path)
)
result = predict(
    model,
    transforms,
    image_path,
    preprocessed_image_path,
)
# result

In [9]:
index = 1
df.loc[index, "embedding"], pd.isna(df.loc[index, "embedding"])

([], array([], dtype=bool))

In [10]:
save_path = "./df_embedding.pkl"

if os.path.exists(save_path):
    print("Load success df_embedding.pkl")
    df = pd.read_pickle(save_path)

Load success df_embedding.pkl


In [11]:
store_id_list = df["store_id"]
image_folder = "/Users/user/Documents/Coding/geo/notebook/data/full_image"
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    store_id = df.loc[index, "store_id"]
    # print(store_id)

    image_path = os.path.join(image_folder, f"{store_id}.png")
    if not os.path.exists(image_path):
        continue
    # is nan or not
    # print('df.loc[index, "embedding"]', df.loc[index, "embedding"])
    if df.loc[index, "embedding"] != []:
        continue
    preprocessed_image_path = os.path.join(
        preprocessed_image_folder, os.path.basename(image_path)
    )
    if os.path.exists(preprocessed_image_path):
        continue
    embedding = predict(
        model,
        transforms,
        image_path,
        preprocessed_image_path,
    )
    df.embedding[index] = embedding.tolist()

Processing rows:   0%|          | 0/11055 [00:00<?, ?it/s]

Processing rows:  23%|██▎       | 2588/11055 [00:00<00:00, 25846.93it/s]/var/folders/9g/3rr3k99j0td2974k71sy46_h0000gp/T/ipykernel_31480/979085148.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.embedding[index] = embedding.tolist()
Processing rows: 100%|██████████| 11055/11055 [1:03:47<00:00,  2.89it/s]


In [12]:
# save df to pickle
df.to_pickle(save_path)